In [1]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
from multiprocessing import Process

## IMPORTING AND SORTING THE DATA

WE HAVE CHOSEN ICECUBE 2008-18 DATA FOR THIS STUDY


THE ICE CUBE DATA SET HAS 1134450 NEUTRINO EVENTS


We select neutrino events with Energy >= 100TeV = 10^5 GeV

i.e log10(E/GeV) > 5


There are 192107 such neutrino events in this data

The ms pulsars are taken from the ATNF Catalogue

There are 441 pulsars (as of May 2022 when the study started)

All the pulsars lie in the declination range of -87 to +87 degrees

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

icdata = icdata.sort_values('log10(E/GeV)')
icdata = icdata.reset_index()
icdata = icdata.drop('index', axis=1)
icdata2 = icdata[icdata['log10(E/GeV)'] > 5]
icdata2 = icdata2.reset_index()
icdata2 = icdata2.drop('index', axis=1)
icdata2

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
mspdata = mspdata.sort_values('DECJD')
mspdata.dropna(inplace=True)
mspdata = mspdata.reset_index()
mspdata = mspdata.drop(["index", "#"], axis=1)

In [3]:
icdata2

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,58192.76586876,5.01,0.20,304.076,-27.773,234.673,62.284
1,57692.17772634,5.01,0.30,328.288,-24.485,225.360,65.593
2,56499.25268749,5.01,0.20,215.304,-72.162,269.459,17.774
3,55742.58579947,5.01,0.29,121.980,-37.347,97.098,52.613
4,57168.08498961,5.01,0.65,209.099,-81.386,154.309,8.532
...,...,...,...,...,...,...,...
192102,57850.41489986,7.00,1.48,299.091,-19.242,135.893,70.801
192103,56616.28975047,7.01,1.24,300.952,-48.575,312.569,41.469
192104,57741.31320782,7.04,3.27,226.680,-7.559,64.185,82.379
192105,58099.46879008,7.26,3.32,317.013,-34.929,22.825,55.146


In [4]:
mspdata

,NAME,Gl,Gb,RAJD,DECJD,P0,F0,DIST
0,J1853-0008g,33.014,-0.453,283.30,-0.13,0.00282,354.609929,4.554
1,J1625-0021,13.890,31.827,246.2931579,-0.358044,0.0028336138772234,352.90623328676,0.951
2,J1852-0044g,32.395,-0.560,283.11,-0.73,0.00241,414.937759,4.492
3,J1835-0114,29.990,3.007,278.841325,-1.242669,0.005116387644239,195.4503977286,3.452
4,J1901-0125,32.817,-2.902,285.39083,-1.42472,0.00279,358.422939,2.360
...,...,...,...,...,...,...,...,...
436,J1944+0907,47.160,-7.357,296.03887491,9.12306353,0.005185201908798642,192.8565208431179,1.218
437,J0023+0923,111.383,-52.849,5.82032291,9.38996121,0.003050203104754390,327.8470205611185,1.818
438,B1855+09,42.290,3.060,284.401626217,9.721442158,0.005362100549682627,186.4940783438289,1.200
439,J2234+0944,76.280,-40.438,338.69522573,9.74172845,0.003627027895734199,275.7078326240928,1.587


In [5]:
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
extensions = 441 - len(icdata2)%441
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icang = [float(i) for i in icdata2['AngErr[deg]']]
icra.extend([0]*extensions)
icdec.extend([0]*extensions)


Selecting events which are at an angular distance < $20^\circ$ from a particular ms psr source = $N$

In [6]:
def hvovec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return np.abs(np.rad2deg(np.arccos(a)))

In [10]:
#all_angles = load.t2a2b(icra,icdec,msra,msdec,extensions,which='a')
all_angles_o = []
op_async = []
pool = []
all_angles_o = []
lg = int(len(icra)/len(msra))
p = len(msra)  
#for k in range(lg):
k = list(range(0,lg))
def mul_angle_finder(k):
    st2a = []
    ilo = icra[k * p  :p * k + p]
    ila = icdec[k * p  :p * k + p]
    lo =[]
    la = []
    for j in range(p):#441
        lo = [msra[(i + j)%p] for i in range(0,p)]
        la = [msdec[(i + j)%p] for i in range(0,p)]
        temp = hvovec(lo, la, ilo, ila)
        
        if k == lg - 1:
            for l in range(extensions, len(temp)):
                temp[l] = -1
        st2a.append(temp)
    return st2a
pool = mul.Pool()
op_async = pool.map_async(mul_angle_finder, range(0,lg))
all_angles_o = op_async.get()

In [25]:
l=0
angl = []
for i in range(len(msra)):
    temp = []
    for k in range(len(all_angles_o)):
        t2 = []
        for j in range(len(all_angles_o[k])):
            if all_angles_o[k][j][((+i-j) % p)] != -1:
                t2.append(all_angles_o[k][j][((+i-j) % p)])
        t3 = [t2.pop(0)]
        t2.reverse()
        t4 = [t2[(l+i)%(len(t2) - 1)] for l in range(len(t2) - 1)]
        t3.extend(t4)

            #temp.append(all_angles_o[k][j][((441+i-j) % p)])
        #t2 = [t2[(i+l)%len(t2)] for l in range(0,len(t2))]
        temp.append(t3)
    angl.append(temp)



In [26]:
len(angl[9][-1])

168

In [ ]:
for i in range(len(angl)):
    for j in range(len(i)):
        

In [27]:
#i=np.random.randint(0,len(msra))
i = 2
print([msra[i], msdec[i]])

[283.11, -0.73]


In [28]:
i

2

In [51]:
len(all_angles_o[i][i])

441

In [42]:
#j=np.random.randint(0,len(icra))
j = 4406
aaaa = hvovec(icra[j],icdec[j], msra[i], msdec[i])
aaaa
j

4406

In [43]:
[j//441,j%441]

[9, 437]

In [44]:
angl[i][j//p].index(aaaa)

433

In [114]:
aaaa

105.56585337585939

In [116]:
for i in range(len(angl)):
    try:
        for j in range(len(angl[i])):    
            print([i,j,angl[i][j].index(aaaa)])
    except ValueError:
        pass

In [ ]:
ss=1
for j in range(169):
    print(hvovec(icra[-441*ss -169+j],icdec[-441*ss -169+j], msra[i], msdec[i]))

In [ ]:
for i in angl:
    a = i.count(None)
    if a != 0:
        print(a)

In [ ]:
len(angl[i])

### all_angles_o[k][j][$(441+i-j-1) \% 441$] = space angle between $p_i$  and $\nu_{kp+j}$


Selecting events which are at an angular distance < $20^\circ$ from a particular ms psr source = $N$

### $S_{ij}$ = $f( \nu$ event $i$ and  pulsar source $j)$

In [ ]:
def S_ij(i,j):
    ang = hvovec(icra[i], icdec[i], msra[j], msdec[j])
    if ang < 20:
        sg = icang[i] ** 2
        return np.exp(-1 * ang / (2 * sg)) / (2 * np.pi * sg)
    else:
        return -1

In [ ]:
S_ij(1,1)

In [ ]:
S = []
#for j in range(p):
def multi_si(j):
    aa = []
    for i in range(len(icdata2)):
        sij = S_ij(i, j)
        #if sij != -1:
        aa.append(sij)
    return aa

pool = mul.Pool()
op_async = pool.map_async(multi_si, range(0,p))
S = op_async.get()

In [ ]:
for i in S:
    print(max(S))

$B_i$ = 103.12813

For each sample, given the Dec δi of an event:

The background PDF is determined by the relative number of
events in δi ± 3◦ divided by the solid angle.


so calculate total no of events within delta +/- 3
and then divide by 2 pi (sin[delta+3]-sin[delta-3])

Solid angle span = $2\pi\cos(\sin(\delta + 6) - sin(\delta - 6))$

We estimate the background PDF by calculating the ratio of average number of events per steradian at each
declination and the total number of events. 

We use the same unbinned likelihood method as in ~\cite{Hooper,Kamionkowski,LuoZhang} which was first proposed in ~\cite{Montaruli10}. Similar to ~\cite{LuoZhang}, we only select those neutrino events, whose angular distance  is within $20^{\circ}$ of a MSP.  For a dataset of $N$ events, if $n_s$ signal events are attributed to a millisecond pulsar, the probability density of an individual event $i$ is given by:
\begin{equation}
P_i = \frac{n_s}{N} S_i + (1-\frac{n_s}{N}) B_i,
%\label{eq1}
\end{equation}
where $S_i$ and $B_i$ represent the signal and background pdfs, respectively.
The likelihood function ($\mathcal{L}$) of the entire dataset, obtained from the product of each individual PDF can be written as 
\begin{equation}
\mathcal{L} (n_s) = \prod_{i=1}^N P_i,
\end{equation}
where $P_i$ is the same as in Eq.~\ref{eq1}. The signal PDF is given by
\begin{equation}
S_i = \frac{1}{2\pi\sigma_i^2}e^{-(|\theta_i-\theta_s|)^2/2\sigma_i^2}
\end{equation}
where $|\theta_i-\theta_s|$ is the angular distance between the  MSP and the neutrino, whereas $\sigma_i$ is the angular uncertainty in the neutrino position.
The background PDF is determined from the average number of events per solid angle  within a declination of $\pm 3^{\circ}$ after averaging over RA. We do not incorporate  the energy information, since the pubic IceCube catalog only contains the energy of the reconstructed muon. The detection statistic used to ascertain the presence of a signal is given by:
\begin{equation}
TS (n_s) = 2 \log \frac{\mathcal{L} (n_s)}{\mathcal{L} (0)}
\end{equation}
 If the null hypothesis is true, $TS (n_s)$ obeys the $\chi^2$ distribution for one degree of freedom. We calculated $TS (n_s)$ for all MSPs in the  catalog.

In [ ]:
def b_ivec(lon1, lat1, lon2, lat2):
  
    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)


    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return (np.rad2deg(np.arccos(a)), a)

In [ ]:
evts_p_srad = []
l_ic = len(icdata2)
for i in range(len(mspdata)):
#returns $B_i$ as defined above
#def B_i(i):

    r = []
    psra = float(mspdata['RAJD'][i])
    psdec = float(mspdata['DECJD'][i])
    pssa = np.cos(np.cos(np.deg2rad(psdec) + np.cos(np.deg2rad(3))) - np.cos(np.deg2rad(psdec) - np.cos(np.deg2rad(3))))
    for j in range(l_ic):
        if icdec[j] > np.deg2rad(psdec) - pssa and icdec[j] < np.deg2rad(psdec) + pssa:
            r.append(j)
    evts_p_srad.append([i,r])

    

n_s = Signal Events from point source

In [ ]:
def Pr(s, S, B, N, ns):
    return ns * S / N + (1 - ns/N)*B
#for i in N
# find P[i]

In [ ]:
def L_i(s, S, B, N, ns):
    L = 1
    for i in range(len(N)):
        L *= Pr(s, S[i], B[i],  N[i], ns)
    return L

In [ ]:
def TS(s, S, B, N):
    ts = []
    for i in range(0,4):
        ts.append(2 * np.log10(L_i(s, S, B, N, i)/L(s, S, B, N, 0)))
    return ts

In [ ]:
#Find ns such that TS is maximized